In [1]:
import tensorflow as tf
import numpy as np
from data_manipulation.data import Data
from data_manipulation.utils import survival_5
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
import tensorflow.keras.optimizers as optimizers
from tensorflow.python.keras import backend as K

In [2]:
height = 224
width = height
dataset_path = '/Users/adalbertoclaudioquiros/Documents/Code/UofG/PhD/Cancer_TMA_Generative'
dataset = 'NKI'
marker = 'HE'

epochs = 40
batch_size = 128
num_classes = 2

data = Data(dataset='nki', marker='he', patch_h=height, patch_w=width, n_channels=3, batch_size=batch_size, project_path=dataset_path)

In [3]:
with tf.Graph().as_default():
    input_shape = data.training.shape[1:]
    input_labels = tf.placeholder(tf.float32, (None, 2), name='labels')
    inception_v3_no_top = inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    net = GlobalAveragePooling2D()(inception_v3_no_top.output)
    logits = Dense(num_classes, activation=None)(net)
    predictions = tf.nn.softmax(logits)
    inception_v3_model = Model(inputs=inception_v3_no_top.input, outputs=logits)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=input_labels, logits=logits))
    train_inception = tf.train.AdamOptimizer(0.002).minimize(loss)
    
    print_epochs = 10
    run_epochs = 0 
    with tf.keras.backend.get_session() as sess:
        K.set_session(sess)
        sess.run(tf.global_variables_initializer())
        for epoch in range(1, epochs+1):
            for batch_images, batch_labels in data.training:
                batch_labels = np.reshape(batch_labels[:, 0], (-1, 1))
                batch_labels = survival_5(batch_labels)
                batch_labels = tf.keras.utils.to_categorical(y=batch_labels, num_classes=2)
                feed_dict = {inception_v3_model.input:batch_images, input_labels:batch_labels}
                sess.run(train_inception, feed_dict)
                # Print losses and Generate samples.
                if run_epochs % print_epochs == 0:
                    epoch_loss = sess.run(loss, feed_dict)
                    print('Loss:', epoch_loss)
                run_epochs += 1
                print(run_epochs)

Loss: 52.088364
1
2
3
4
5
6
7
8
9
10
Loss: 0.6210083
11
12
13
14
15
16


KeyboardInterrupt: 